In [5]:
import numpy as np
import sklearn
import matplotlib.pyplot as plt
import csv
import gc

from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import r2_score, accuracy_score, f1_score, precision_score, recall_score
from sklearn.model_selection import train_test_split

In [6]:
def normalization(Xtrain, Xtest):
    print('Data standardization to zero mean and unit variance...')
    scaler = StandardScaler()
    Xstandardized_train = scaler.fit_transform(Xtrain)
    Xstandardized_test = scaler.transform(Xtest)
    return Xstandardized_train, Xstandardized_test

def datacsvreadTE(filename, delimiter='\t', verbose=False):
    if verbose:
        print('===> datacsvreadTE> Reading TE data from file ', filename, '...')

    with open(filename, 'rt') as f:
        reader = csv.reader(f, delimiter=delimiter)
        row1 = next(reader)
        nfeat = len([cell for cell in row1 if cell != ''])
        X = []
        for row in reader:
            x = [float(cell) for cell in row if cell != '']
            if len(x) == nfeat:
                X.append(x)
    return np.array(X)

def read_train_test_pair(datadir, cond_num='01', fault_start=160, standardize=True):
    assert datadir is not None, 'TE data dir must not be empty'

    ftrain = datadir + 'd' + cond_num + '.dat'
    ftest = datadir + 'd' + cond_num + '_te.dat'
    
    if ftrain == datadir + 'd00.dat':
        Xtrain = datacsvreadTE(ftrain, delimiter='\t')
    else:
        Xtrain = datacsvreadTE(ftrain, delimiter=' ')

    Xtest = datacsvreadTE(ftest, delimiter=' ')

    if standardize:
        Xtrain, Xtest = normalization(Xtrain, Xtest)
    
    return Xtrain, Xtest

def print_scores(y_test_overall, y_pred_overall, one_VS_one=True):
    accuracy = 100 * accuracy_score(y_test_overall, y_pred_overall)
    precision = [precision_score(y_test_overall, y_pred_overall, average=i, zero_division=True) for i in ['macro', 'micro']]
    recall = [recall_score(y_test_overall, y_pred_overall, average=i, zero_division=True) for i in ['macro', 'micro']]
    f1 = [f1_score(y_test_overall, y_pred_overall, average=i, zero_division=True) for i in ['macro', 'micro']]

    filename = "Results_OneVsOne.txt" if one_VS_one else "Results_All.txt"
    with open(filename, "a") as file:
        print(f'Classification Report =\n{classification_report(y_test_overall, y_pred_overall, zero_division=1)}', file=file)
        print(f'Accuracy= {accuracy:.3f} %', file=file)
        print(f'Macro-averaged Precision = {precision[0]:.3f} %', file=file)
        print(f'Micro-averaged Precision = {precision[1]:.3f} %', file=file)
        print(f'Macro-averaged Recall = {recall[0]:.3f} %', file=file)
        print(f'Micro-averaged Recall = {recall[1]:.3f} %', file=file)
        print(f'Macro-averaged F1 = {f1[0]:.3f} %', file=file)
        print(f'Micro-averaged F1 = {f1[1]:.3f} %', file=file)
        print("=" * 15, file=file)

rootdir = 'Downloads/'  
datadir = rootdir + 'TE_process/data/'          

In [7]:
def classify_one_against_one(condition_classes, hidden_layer_sizes=(100, 50), max_iter=1000):
    fault_start = 160
    cc1 = '00'
    print(f"condition_classes: {condition_classes}")
    
    Xtrain, Xtest = read_train_test_pair(datadir, cond_num=cc1, standardize=False)
    traindata1 = Xtrain
    trainlabels1 = np.zeros(len(traindata1), dtype=int)
    testdata1 = Xtest[fault_start:]
    testlabels1 = np.zeros(len(testdata1), dtype=int)

    for j in range(1, len(condition_classes)):
        cc2 = condition_classes[j]
        print('Class', cc1, 'against', cc2)
        
        Xtrain, Xtest = read_train_test_pair(datadir, cond_num=cc2, standardize=False)
        traindata2 = Xtrain
        testdata2 = Xtest[fault_start:]
        trainlabels2 = np.ones(len(traindata2), dtype=int)
        testlabels2 = np.ones(len(testdata2), dtype=int)
        
        X = np.vstack((traindata1, traindata2, testdata1, testdata2))
        y = np.concatenate((trainlabels1, trainlabels2, testlabels1, testlabels2))

        X = StandardScaler().fit_transform(X)

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

        model = MLPClassifier(hidden_layer_sizes=hidden_layer_sizes, max_iter=max_iter, 
                              early_stopping=True, random_state=42, learning_rate_init=0.001)
        model.fit(X_train, y_train)

        y_pred = model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)

        with open("Results_OneVsOne.txt", "a") as file:
            print(f"'Class', {cc1}, 'against', {cc2} -> TestAcc: {accuracy:.4f}", file=file)

        print_scores(y_test, y_pred)

        gc.collect()

In [8]:
def classify_all(condition_classes, hidden_layer_sizes=(100, 50), max_iter=1000):
    fault_start = 160
    print(f"condition_classes: {condition_classes}")

    X = []
    y = []

    for i, cc in enumerate(condition_classes):
        Xtrain, Xtest = read_train_test_pair(datadir, cond_num=cc, standardize=False)
        X.extend(Xtrain)
        X.extend(Xtest[fault_start:])
        y.extend(i * np.ones(len(Xtrain), dtype=int))
        y.extend(i * np.ones(len(Xtest[fault_start:]), dtype=int))
    
    X = np.array(X)
    y = np.array(y)

    X = StandardScaler().fit_transform(X)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

    model = MLPClassifier(hidden_layer_sizes=hidden_layer_sizes, max_iter=max_iter, 
                          early_stopping=True, random_state=42, learning_rate_init=0.001)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)

    with open("Results_All.txt", "a") as file:
        print(f"'All' -> TestAcc: {accuracy:.4f}", file=file)

    print_scores(y_test, y_pred, one_VS_one=False)

    gc.collect()

In [9]:
all_fault_num = [str(i).zfill(2) for i in range(1, 22)]
condition_classes = ['00'] + all_fault_num

for fault in all_fault_num:
    one_vs_one = ['00', fault]
    classify_one_against_one(one_vs_one, hidden_layer_sizes=(100, 50), max_iter=1000)

classify_all(condition_classes, hidden_layer_sizes=(100, 50), max_iter=1000)

condition_classes: ['00', '01']
Class 00 against 01
condition_classes: ['00', '02']
Class 00 against 02
condition_classes: ['00', '03']
Class 00 against 03
condition_classes: ['00', '04']
Class 00 against 04
condition_classes: ['00', '05']
Class 00 against 05
condition_classes: ['00', '06']
Class 00 against 06
condition_classes: ['00', '07']
Class 00 against 07
condition_classes: ['00', '08']
Class 00 against 08
condition_classes: ['00', '09']
Class 00 against 09
condition_classes: ['00', '10']
Class 00 against 10
condition_classes: ['00', '11']
Class 00 against 11
condition_classes: ['00', '12']
Class 00 against 12
condition_classes: ['00', '13']
Class 00 against 13
condition_classes: ['00', '14']
Class 00 against 14
condition_classes: ['00', '15']
Class 00 against 15
condition_classes: ['00', '16']
Class 00 against 16
condition_classes: ['00', '17']
Class 00 against 17
condition_classes: ['00', '18']
Class 00 against 18
condition_classes: ['00', '19']
Class 00 against 19
condition_cl